# PEM key format

We can generate key in PEM format using e.g. OpenSSL:  

```bash
openssl genpkey -algorithm RSA -out private_key.pem -pkeyopt rsa_keygen_bits:4096
```

Such generated key would be called `RSA-4096`, where 4096 is number of bits in the modulus.



In [2]:
from cryptography.hazmat.primitives import serialization

with open('private_key.pem', 'rb') as key:
    private_key = serialization.load_pem_private_key(
        key.read(),
        password=None
    )
# private numbers
d = Integer(private_key.private_numbers().d)
p = Integer(private_key.private_numbers().p)
q = Integer(private_key.private_numbers().q)
# public numbers
e = Integer(private_key.private_numbers().public_numbers.e)
n = Integer(private_key.private_numbers().public_numbers.n)
print(f"d = {d}\nq = {q}\np = {p}\nn = p*q = {p * q}")
print(f"e = {e}")
print(f"p has {p.ndigits()} digits ({p.nbits()} bits), q has {q.ndigits()} digits ({q.nbits()} bits), n has {n.ndigits()} digits ({n.nbits()} bits)")

d = 337437341741734694491726362505547873764305515643827607185066575838005364744697882803784218778084478977846005006716635279247490723297691397961784754290476998182403654519547802243937847525487311019367043874372518865913244019582575016040699384210605266693680917375599827709712296493283072762145506545684969822897365069181468490059185740275877104025199985233405538445728229840709439990858210888669657665288137990327426905965355120438087129270659359963488573620380313993140990617827119441222509263069179070688414552312414419446403454039341497306572323721163902662838622254793361675022803315954572094608574607716741074528427448318513166309535630142244739896974414106330877950294508589328081647867532781928418248196629934618718197369576388989281771931186468215113673309793684356898390729694103607312005531661755342253865223430310664021821399870959244859129223158343511685606619862463844804909560064942244789185938694258385174949771119434921738274779181342159400251743451118815904244782630303598907279253

# Encryption and decryption

Using RSA equation of form  $(m^e)^d \equiv m \mod n$ lets encrypt a value of $m=22$.



In [3]:
m = 22
m

22

In [4]:
temp = m ** e
temp

...

In [6]:
#temp.ndigits()
temp.nbits()

292258

In [7]:
encrypted_m = (m ** e) % n
encrypted_m

2839495989946032472651636246015698701326362084335873954595160823952706347118128420614146100842427404552705816774630998647691852547810146226096591328232455545843702891095458426175477932113374885964558604340313428101430780404964905645672400507645579678125684027013024318624365824211580607955450857181666787285001854783622043751967880193415730902152095824933514300572832451203820151324472470202751957125256901069816107207541731478895040517968653097465217845886299187463874746058934424430512665694000792107496952502891463276011353125737254990000615126969773993308657421274631871828346012245463425826732969590041787802122924658167030144066926523170502329267059119013978338161087382277748560560863350858416568267601373226082942007811120924579970703268804371525797385536086184455709322259548156745285546524842669752047347956025095512069150498101643456490272992251605263176819250390362840571067923692341633161248898990664921074373616872181561102230769405253973596124068967079711305391569935245886213258439851

We can now send public\-key encrypted message over insecure channel to owner of corresponding private key for decryption.

Let's see how decryption works.



In [0]:
# encrypted_m ** d # won't work due to exponent too large for naive power

In [8]:
decrypted_m = power_mod(encrypted_m, d, n)
decrypted_m

22

# Signing and verification

... but wait, there's more.

Asymmetric cryptography can be used for signing and verification, where holder of private key signs a message which authenticity can be later verified using corresponding public key.

Let's create a signature $s$ of the same message $m = 22$ using our private key:

$s \equiv m^d \mod n$



In [0]:
# m ** d # won't work due to exponent too large for naive power

In [10]:
s = power_mod(m, d, n)
s.nbits()

4094

And now verify signature:

$(m^d)^e \mod n \equiv s^e \mod n$


In [11]:
temp_s = s ** e # might not work due to running out of memory
temp_s.ndigits()

80755629

In [12]:
power_mod(s, e, n)

22

_Reminder to switch sharing to other tab._
